In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

Using TensorFlow backend.


# Generator

In [3]:
def build_generator(latent_dim, channels):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

# Discriminator

In [4]:
def build_discriminator(img_shape):
    """ CNN is defined manually inside this procedure
        using Leaky RELU
    """

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

# save figures in ./images

In [5]:
def save_imgs(epoch, latent_dim, G):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

# training the model

In [6]:
def train(D, G, combined, epochs, latent_dim, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = G.predict(noise)

        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = D.train_on_batch(imgs, valid)
        d_loss_fake = D.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = combined.train_on_batch(noise, valid)

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch, latent_dim, G)

# main()

In [8]:
# Input shape
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

# optimizer

In [9]:
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# Build and compile the discriminator

In [10]:
D = build_discriminator(img_shape)
D.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)         

In [11]:
# Build the generator
G = build_generator(latent_dim, channels)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 128)      

In [12]:
# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = G(z)

In [13]:
# For the combined model we will only train the generator
G.trainable = False

In [16]:
# The discriminator takes generated images as input and determines validity
valid = D(img)

In [17]:
# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [18]:
epochs=4000

In [ ]:
train(D, G, combined, 
      latent_dim=latent_dim,
      epochs=epochs, 
      batch_size=32, save_interval=50)


0 [D loss: 0.971818, acc.: 39.06%] [G loss: 0.904005]
1 [D loss: 0.580782, acc.: 68.75%] [G loss: 0.640387]
2 [D loss: 0.520761, acc.: 67.19%] [G loss: 0.575532]
3 [D loss: 0.329176, acc.: 81.25%] [G loss: 0.501298]
4 [D loss: 0.399877, acc.: 71.88%] [G loss: 0.459840]
5 [D loss: 0.287807, acc.: 85.94%] [G loss: 0.242169]
6 [D loss: 0.170066, acc.: 96.88%] [G loss: 0.171092]
7 [D loss: 0.154485, acc.: 92.19%] [G loss: 0.120855]
8 [D loss: 0.095455, acc.: 96.88%] [G loss: 0.154423]
9 [D loss: 0.062577, acc.: 100.00%] [G loss: 0.091144]
10 [D loss: 0.077430, acc.: 96.88%] [G loss: 0.089175]
11 [D loss: 0.057203, acc.: 96.88%] [G loss: 0.076569]
12 [D loss: 0.049290, acc.: 100.00%] [G loss: 0.080518]
13 [D loss: 0.032268, acc.: 100.00%] [G loss: 0.039273]
14 [D loss: 0.024370, acc.: 100.00%] [G loss: 0.024534]
15 [D loss: 0.029122, acc.: 100.00%] [G loss: 0.020207]
16 [D loss: 0.016629, acc.: 100.00%] [G loss: 0.020623]
17 [D loss: 0.008096, acc.: 100.00%] [G loss: 0.013124]
18 [D loss: 

141 [D loss: 0.000158, acc.: 100.00%] [G loss: 0.000576]
142 [D loss: 0.000451, acc.: 100.00%] [G loss: 0.000298]
143 [D loss: 0.000228, acc.: 100.00%] [G loss: 0.000464]
144 [D loss: 0.000297, acc.: 100.00%] [G loss: 0.000528]
145 [D loss: 0.000340, acc.: 100.00%] [G loss: 0.000799]
146 [D loss: 0.000310, acc.: 100.00%] [G loss: 0.000475]
147 [D loss: 0.000221, acc.: 100.00%] [G loss: 0.000498]
148 [D loss: 0.000157, acc.: 100.00%] [G loss: 0.000278]
149 [D loss: 0.000139, acc.: 100.00%] [G loss: 0.000781]
150 [D loss: 0.000209, acc.: 100.00%] [G loss: 0.000379]
151 [D loss: 0.000195, acc.: 100.00%] [G loss: 0.000318]
152 [D loss: 0.000145, acc.: 100.00%] [G loss: 0.000259]
153 [D loss: 0.000465, acc.: 100.00%] [G loss: 0.000259]
154 [D loss: 0.000167, acc.: 100.00%] [G loss: 0.000723]
155 [D loss: 0.000122, acc.: 100.00%] [G loss: 0.000239]
156 [D loss: 0.000312, acc.: 100.00%] [G loss: 0.000543]
157 [D loss: 0.000268, acc.: 100.00%] [G loss: 0.000446]
158 [D loss: 0.000326, acc.: 10